In [121]:
pip install meteostat geopy

Note: you may need to restart the kernel to use updated packages.


In [122]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

state: US state, district, or territory.
declaration_type: One of "DR" (= major disaster), "EM" (= emergency management), or "FM" (= "fire management")
declaration_date: Date the disaster was declared.
fy_declared: Fiscal year in which the disaster was declared.
incident_type: Type of incident such as "Fire", "Flood", or "Hurricane". The incident type will affect the types of assistance available.
declaration_title: Title for the disaster. This can be a useful identifier such as "Hurricane Katrina" or "Covid-19 Pandemic".

In [124]:
data=pd.read_csv('C:/Users/Phantom/Documents/AI/FloodGuard/us_disaster_declarations.csv')
data.tail()

,fema_declaration_string,disaster_number,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,ih_program_declared,ia_program_declared,...,incident_end_date,disaster_closeout_date,fips,place_code,designated_area,declaration_request_number,last_ia_filing_date,last_refresh,hash,id
64087,DR-4696-ME,4696,ME,DR,2023-03-22T00:00:00Z,2023,Severe Storm,Severe Storm And Flooding,0,0,...,2022-12-24T00:00:00Z,NaN,23031,99031,"York (County)(in MSA 6400,6450)",23013,NaN,2023-03-23T22:31:41Z,c6ed4ead7fd3bee8113ef15d9110e845c56a8b2f,3bd84f66-74d0-4e4a-9287-2414fda89014
64088,DR-4697-MS,4697,MS,DR,2023-03-26T00:00:00Z,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",1,0,...,2023-03-25T00:00:00Z,NaN,28015,99015,Carroll (County),23025,2023-05-25T00:00:00Z,2023-03-26T06:41:23Z,9ccd8b25b2e64d4e44a043b1da61f3c42e2d02a9,764fa617-efd9-44c4-9d4a-c48e972b485c
64089,DR-4697-MS,4697,MS,DR,2023-03-26T00:00:00Z,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",1,0,...,2023-03-25T00:00:00Z,NaN,28053,99053,Humphreys (County),23025,2023-05-25T00:00:00Z,2023-03-26T06:41:23Z,62d6f324279680dbb6b1eb2ce91a9bf5ae964e24,c9300696-062b-430f-9f9d-3ad6b1562779
64090,DR-4697-MS,4697,MS,DR,2023-03-26T00:00:00Z,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",1,0,...,2023-03-25T00:00:00Z,NaN,28095,99095,Monroe (County),23025,2023-05-25T00:00:00Z,2023-03-26T06:41:23Z,5cb6b2789fc77375f0a1d0e1a8898a1ce3dfc952,df697ed2-4c59-4d02-84e7-d4e84d6e2723
64091,DR-4697-MS,4697,MS,DR,2023-03-26T00:00:00Z,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",1,0,...,2023-03-25T00:00:00Z,NaN,28125,99125,Sharkey (County),23025,2023-05-25T00:00:00Z,2023-03-26T06:41:23Z,ef1b370fa2c218a02e815935fc7a35173f266e30,821952eb-f0a2-4109-9fa3-b7029e882874


In [125]:
data=data.drop(["fema_declaration_string","disaster_number","pa_program_declared","hm_program_declared","incident_begin_date","incident_end_date","fips","declaration_request_number","ih_program_declared","ia_program_declared","place_code","last_ia_filing_date","last_refresh","hash","id","disaster_closeout_date"],axis=1)
data.head()

,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,designated_area
0,GA,DR,1953-05-02T00:00:00Z,1953,Tornado,Tornado,Statewide
1,TX,DR,1953-05-15T00:00:00Z,1953,Tornado,Tornado & Heavy Rainfall,Statewide
2,LA,DR,1953-05-29T00:00:00Z,1953,Flood,Flood,Statewide
3,MI,DR,1953-06-02T00:00:00Z,1953,Tornado,Tornado,Statewide
4,MT,DR,1953-06-06T00:00:00Z,1953,Flood,Floods,Statewide


In [126]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64092 entries, 0 to 64091
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   state              64092 non-null  object
 1   declaration_type   64092 non-null  object
 2   declaration_date   64092 non-null  object
 3   fy_declared        64092 non-null  int64 
 4   incident_type      64092 non-null  object
 5   declaration_title  64092 non-null  object
 6   designated_area    64092 non-null  object
dtypes: int64(1), object(6)
memory usage: 3.4+ MB


# Removing Time from date column

In [127]:
data['declaration_date']=data['declaration_date'].apply(lambda x: x[:10])

In [128]:
data.head()

,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,designated_area
0,GA,DR,1953-05-02,1953,Tornado,Tornado,Statewide
1,TX,DR,1953-05-15,1953,Tornado,Tornado & Heavy Rainfall,Statewide
2,LA,DR,1953-05-29,1953,Flood,Flood,Statewide
3,MI,DR,1953-06-02,1953,Tornado,Tornado,Statewide
4,MT,DR,1953-06-06,1953,Flood,Floods,Statewide


In [129]:
data['incident_type'].nunique()

25

In [130]:
data['incident_type'].unique()

array(['Tornado', 'Flood', 'Fire', 'Other', 'Earthquake', 'Hurricane',
       'Volcanic Eruption', 'Severe Storm', 'Toxic Substances', 'Typhoon',
       'Drought', 'Dam/Levee Break', 'Snowstorm', 'Severe Ice Storm',
       'Freezing', 'Coastal Storm', 'Fishing Losses', 'Mud/Landslide',
       'Human Cause', 'Terrorist', 'Tsunami', 'Chemical', 'Biological',
       'Tropical Storm', 'Winter Storm'], dtype=object)

In [131]:
data.tail(10)

,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,designated_area
64082,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,Franklin (County)
64083,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,Knox (County)
64084,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,Oxford (County)
64085,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,Somerset (County)
64086,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,Waldo (County)
64087,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,"York (County)(in MSA 6400,6450)"
64088,MS,DR,2023-03-26,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Carroll (County)
64089,MS,DR,2023-03-26,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Humphreys (County)
64090,MS,DR,2023-03-26,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Monroe (County)
64091,MS,DR,2023-03-26,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Sharkey (County)


# Renaming the column titles

In [132]:
data=data.rename(columns={'declaration_type': 'aloikatte','declaration_date':'date','fy_declared':'year','incident_type':'disaster','declaration_title':'disaster_info'})

In [133]:
data.head()

,state,aloikatte,date,year,disaster,disaster_info,designated_area
0,GA,DR,1953-05-02,1953,Tornado,Tornado,Statewide
1,TX,DR,1953-05-15,1953,Tornado,Tornado & Heavy Rainfall,Statewide
2,LA,DR,1953-05-29,1953,Flood,Flood,Statewide
3,MI,DR,1953-06-02,1953,Tornado,Tornado,Statewide
4,MT,DR,1953-06-06,1953,Flood,Floods,Statewide


In [134]:
data=data.drop(['year'],axis=1)

In [135]:
data=data.rename(columns={'aloikatte':'category'})

In [136]:
def check_severity(row):
    if 'severe' in row['disaster'].lower() and 'severe' in row['disaster_info'].lower():
        return 1
    elif ('severe' in row['disaster'].lower() or 'severe' in row['disaster_info'].lower()) and not ('severe' in row['disaster'].lower() and 'severe' in row['disaster_info'].lower()):
        return 0.5
    else:
        return 0

data['severity'] = data.apply(check_severity, axis=1)
data.tail(15)

,state,category,date,disaster,disaster_info,designated_area,severity
64077,VT,DR,2023-03-20,Severe Storm,Severe Storm And Flooding,Grand Isle (County),1.0
64078,VT,DR,2023-03-20,Severe Storm,Severe Storm And Flooding,Lamoille (County),1.0
64079,VT,DR,2023-03-20,Severe Storm,Severe Storm And Flooding,Orange (County),1.0
64080,VT,DR,2023-03-20,Severe Storm,Severe Storm And Flooding,Orleans (County),1.0
64081,VT,DR,2023-03-20,Severe Storm,Severe Storm And Flooding,Washington (County),1.0
64082,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,Franklin (County),1.0
64083,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,Knox (County),1.0
64084,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,Oxford (County),1.0
64085,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,Somerset (County),1.0
64086,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,Waldo (County),1.0


In [137]:
data = data.drop_duplicates(subset=['state', 'date', 'disaster','disaster_info','designated_area'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64014 entries, 0 to 64091
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   state            64014 non-null  object 
 1   category         64014 non-null  object 
 2   date             64014 non-null  object 
 3   disaster         64014 non-null  object 
 4   disaster_info    64014 non-null  object 
 5   designated_area  64014 non-null  object 
 6   severity         64014 non-null  float64
dtypes: float64(1), object(6)
memory usage: 3.9+ MB


In [138]:
data.reset_index(drop=True, inplace=True)

In [139]:
data['disaster'].unique()

array(['Tornado', 'Flood', 'Fire', 'Other', 'Earthquake', 'Hurricane',
       'Volcanic Eruption', 'Severe Storm', 'Toxic Substances', 'Typhoon',
       'Drought', 'Dam/Levee Break', 'Snowstorm', 'Severe Ice Storm',
       'Freezing', 'Coastal Storm', 'Fishing Losses', 'Mud/Landslide',
       'Human Cause', 'Terrorist', 'Tsunami', 'Chemical', 'Biological',
       'Tropical Storm', 'Winter Storm'], dtype=object)

In [140]:
data.tail()

,state,category,date,disaster,disaster_info,designated_area,severity
64009,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,"York (County)(in MSA 6400,6450)",1.0
64010,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Carroll (County),1.0
64011,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Humphreys (County),1.0
64012,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Monroe (County),1.0
64013,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Sharkey (County),1.0


# Removing unwanted disaster categories

In [141]:
data=data[~data['disaster'].isin(['Other', 'Chemical', 'Biological', 
                                       'Fishing Losses', 'Dam/Levee Break', 
                                       'Terrorist', 'Human Cause', 
                                       'Toxic Substances'])]
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55839 entries, 0 to 64013
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   state            55839 non-null  object 
 1   category         55839 non-null  object 
 2   date             55839 non-null  object 
 3   disaster         55839 non-null  object 
 4   disaster_info    55839 non-null  object 
 5   designated_area  55839 non-null  object 
 6   severity         55839 non-null  float64
dtypes: float64(1), object(6)
memory usage: 3.4+ MB


# Removing Data before 2000's from date column¶

In [142]:
data.loc[:, 'date'] = pd.to_datetime(data['date'])

# Filter for disasters from the year 2000 onward
data = data[data['date'] >= pd.Timestamp('2000-01-01')].copy()

# Show DataFrame info
data.head()

,state,category,date,disaster,disaster_info,designated_area,severity
19683,TX,FM,2000-01-03 00:00:00,Fire,Saddleback Fire,Wise (County),0.0
19684,TX,FM,2000-01-04 00:00:00,Fire,Purgatory Fire,Comal (County),0.0
19685,TX,FM,2000-01-05 00:00:00,Fire,Bob's Trail Fire,Bastrop (County),0.0
19686,KY,DR,2000-01-10 00:00:00,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Ballard (County),0.5
19687,KY,DR,2000-01-10 00:00:00,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Breckinridge (County),0.5


In [143]:
data.loc[:,'date'] = data['date'].astype(str)
data.loc[:,'date']=data['date'].apply(lambda x: x[:10])

In [144]:
data.head()

,state,category,date,disaster,disaster_info,designated_area,severity
19683,TX,FM,2000-01-03,Fire,Saddleback Fire,Wise (County),0.0
19684,TX,FM,2000-01-04,Fire,Purgatory Fire,Comal (County),0.0
19685,TX,FM,2000-01-05,Fire,Bob's Trail Fire,Bastrop (County),0.0
19686,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Ballard (County),0.5
19687,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Breckinridge (County),0.5


In [145]:
data.reset_index(drop=True, inplace=True)

# Testing Api

In [147]:
location = Point(41.1400, -104.8202, 1800)  # Latitude, Longitude, Elevation in meters

# Set the date range for historical data
start_date = datetime(2020, 6, 4)  # Start date
end_date = datetime(2020, 6, 4)    # End date

# Fetch daily weather data
new = Daily(location, start_date, end_date)
new = new.fetch()

# Check if data is available
if not new.empty:
    print("Weather data for Los Angeles, CA:")
    print(new)
else:
    print("No data available for the specified location and date range.")


Weather data for Los Angeles, CA:
            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2020-06-04  21.2  13.8  30.5   0.0   0.0  275.0  23.0   NaN  1007.8   NaN


In [149]:
data.head()

,state,category,date,disaster,disaster_info,designated_area,severity
0,TX,FM,2000-01-03,Fire,Saddleback Fire,Wise (County),0.0
1,TX,FM,2000-01-04,Fire,Purgatory Fire,Comal (County),0.0
2,TX,FM,2000-01-05,Fire,Bob's Trail Fire,Bastrop (County),0.0
3,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Ballard (County),0.5
4,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Breckinridge (County),0.5


# Using meteostat API to get the climate on the day of disasters

In [236]:
from datetime import datetime
from meteostat import Point, Daily


# Function to get coordinates and elevation for a given state
state_coordinates = {
    'AL': (33.5186, -86.8104, 150),   # Alabama
    'AK': (61.2181, -149.8631, 30),  # Alaska
    'AZ': (33.4484, -112.0740, 340),  # Arizona
    'AR': (34.7465, -92.2896, 100),    # Arkansas
    'CA': (34.0522, -118.2437, 100),   # California
    'CO': (39.7392, -104.9903, 1600),  # Colorado
    'CT': (41.7630, -72.6851, 30),    # Connecticut
    'DE': (39.7392, -75.5398, 30),     # Delaware
    'FL': (25.7617, -80.1918, 2),     # Florida
    'GA': (33.7490, -84.3880, 320),    # Georgia
    'HI': (21.3069, -157.8583, 30),  # Hawaii
    'ID': (43.6150, -116.2023, 800),  # Idaho
    'IL': (41.8781, -87.6298, 180),     # Illinois
    'IN': (39.7684, -86.1581, 200),     # Indiana
    'IA': (41.5868, -93.6250, 300),     # Iowa
    'KS': (39.0997, -94.5786, 300),     # Kansas
    'KY': (38.2527, -85.7585, 150),     # Kentucky
    'LA': (29.9511, -90.0715, 10),      # Louisiana
    'ME': (43.6615, -70.2553, 30),     # Maine
    'MD': (39.2904, -76.6122, 30),     # Maryland
    'MA': (42.2302, -71.5301, 300),     # Massachusetts
    'MI': (43.3266, -84.5361, 300),     # Michigan
    'MN': (44.9778, -93.2650, 260),     # Minnesota
    'MS': (32.2988, -90.1848, 100),     # Mississippi
    'MO': (38.6270, -90.1994, 150),     # Missouri
    'MT': (45.7833, -108.5007, 1000),   # Montana
    'NE': (41.2565, -95.9345, 300),     # Nebraska
    'NV': (36.1699, -115.1398, 600),   # Nevada
    'NH': (42.9956, -71.4548, 150),     # New Hampshire
    'NJ': (40.2171, -74.7429, 60),     # New Jersey
    'NM': (35.0844, -106.6504, 1500),   # New Mexico
    'NY': (40.7128, -74.0060, 10),     # New York
    'NC': (35.7796, -78.6382, 100),     # North Carolina
    'ND': (46.8083, -100.7837, 500),     # North Dakota
    'OH': (39.9612, -82.9988, 300),     # Ohio
    'OK': (35.4676, -97.5164, 300),     # Oklahoma
    'OR': (45.5155, -122.6793, 300),   # Oregon
    'PA': (39.9526, -75.1652, 300),     # Pennsylvania
    'RI': (41.8240, -71.4128, 300),      # Rhode Island
    'SC': (34.0007, -81.0348, 300),     # South Carolina
    'SD': (44.3683, -100.3516, 400),     # South Dakota
    'TN': (36.1627, -86.7816, 300),     # Tennessee
    'TX': (30.2672, -97.7431, 300),     # Texas
    'UT': (40.7608, -111.8910, 1300),   # Utah
    'VT': (44.2601, -72.5754, 500),    # Vermont
    'VA': (37.5407, -77.4360, 150),     # Virginia
    'WA': (47.6062, -122.3321, 100),   # Washington
    'WV': (38.3498, -81.6326, 500),     # West Virginia
    'WI': (43.0731, -89.4012, 300),     # Wisconsin
    'WY': (41.1400, -104.8202, 1800)    # Wyoming
}


def get_weather(row):
    date = row['date']
    state = row['state']

    # Convert date to start and end datetime objects (for daily weather data)
    start = datetime.strptime(date, '%Y-%m-%d')
    end = start  # Single day data

    # Get coordinates for the state
    if state in state_coordinates:
        lat, lon, elev = state_coordinates[state]
        if lat is None or lon is None:
            return pd.Series([None]*11, index=['state','date','designated_area','tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres'])
    else:
        # Handle cases where the state is not in the dictionary
        return pd.Series([None]*11, index=['state','date','designated_area','tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres'])

    # Define the city point using coordinates (elevation is optional, default is 0)
    location = Point(lat, lon, elev or 0)
    new = Daily(location, start, end).fetch()

    # Check if new DataFrame is empty and handle accordingly
    if not new.empty:
        # Use the correct column names based on the DataFrame structure
        weather_data = new.iloc[0]  # First (and only) row of data for the given date
        return pd.Series([
            row.get('state'),
            row.get('date'),
            row.get('designated_area'),
            weather_data.get('tavg'),
            weather_data.get('tmin'),
            weather_data.get('tmax'),
            weather_data.get('prcp'),
            weather_data.get('snow'),
            weather_data.get('wdir'),
            weather_data.get('wspd'),
            weather_data.get('pres')
        ], index=['state','date','designated_area','tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres'])
    else:
        return pd.Series([None]*11, index=['state','date','designated_area','tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres'])

# Apply the get_weather function to the DataFrame
weather_df = data.apply(get_weather, axis=1)

# Display the resulting DataFrame
print(weather_df.head())


    

  state        date        designated_area  tavg  tmin  tmax  prcp  snow  \
0    TX  2000-01-03          Wise (County)  15.5   9.4  21.1   0.0   0.0   
1    TX  2000-01-04         Comal (County)   6.1  -0.6  12.2   0.0   0.0   
2    TX  2000-01-05       Bastrop (County)   6.1  -3.4  15.0   0.0   0.0   
3    KY  2000-01-10       Ballard (County)   8.9   4.4  13.3   0.0   0.0   
4    KY  2000-01-10  Breckinridge (County)   8.9   4.4  13.3   0.0   0.0   

    wdir  wspd    pres  
0  294.0  17.3  1012.8  
1    NaN  13.7  1030.2  
2    NaN   9.0  1025.3  
3  198.0  25.6  1002.9  
4  198.0  25.6  1002.9  


In [237]:
weather_df.head()

,state,date,designated_area,tavg,tmin,tmax,prcp,snow,wdir,wspd,pres
0,TX,2000-01-03,Wise (County),15.5,9.4,21.1,0.0,0.0,294.0,17.3,1012.8
1,TX,2000-01-04,Comal (County),6.1,-0.6,12.2,0.0,0.0,NaN,13.7,1030.2
2,TX,2000-01-05,Bastrop (County),6.1,-3.4,15.0,0.0,0.0,NaN,9.0,1025.3
3,KY,2000-01-10,Ballard (County),8.9,4.4,13.3,0.0,0.0,198.0,25.6,1002.9
4,KY,2000-01-10,Breckinridge (County),8.9,4.4,13.3,0.0,0.0,198.0,25.6,1002.9


# Saving weather and disaster dataset as csv

In [239]:
weather_df.to_csv('C:/Users/Phantom/Documents/AI/FloodGuard/weather.csv', index=False)

In [240]:
data.to_csv('C:/Users/Phantom/Documents/AI/FloodGuard/disaster.csv', index=False)

# Combining weather and disaster datasets

In [242]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36230 entries, 0 to 36229
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   state            34344 non-null  object 
 1   date             34344 non-null  object 
 2   designated_area  34344 non-null  object 
 3   tavg             34133 non-null  float64
 4   tmin             34344 non-null  float64
 5   tmax             34344 non-null  float64
 6   prcp             33792 non-null  float64
 7   snow             28609 non-null  float64
 8   wdir             22691 non-null  float64
 9   wspd             33972 non-null  float64
 10  pres             33251 non-null  float64
dtypes: float64(8), object(3)
memory usage: 3.0+ MB


In [243]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36230 entries, 0 to 36229
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   state            36230 non-null  object 
 1   category         36230 non-null  object 
 2   date             36230 non-null  object 
 3   disaster         36230 non-null  object 
 4   disaster_info    36230 non-null  object 
 5   designated_area  36230 non-null  object 
 6   severity         36230 non-null  float64
dtypes: float64(1), object(6)
memory usage: 1.9+ MB


In [252]:
df = pd.merge(data, weather_df, on=['state', 'date','designated_area'], how='inner')

In [254]:
df.head(40)

,state,category,date,disaster,disaster_info,designated_area,severity,tavg,tmin,tmax,prcp,snow,wdir,wspd,pres
0,TX,FM,2000-01-03,Fire,Saddleback Fire,Wise (County),0.0,15.5,9.4,21.1,0.0,0.0,294.0,17.3,1012.8
1,TX,FM,2000-01-04,Fire,Purgatory Fire,Comal (County),0.0,6.1,-0.6,12.2,0.0,0.0,NaN,13.7,1030.2
2,TX,FM,2000-01-05,Fire,Bob's Trail Fire,Bastrop (County),0.0,6.1,-3.4,15.0,0.0,0.0,NaN,9.0,1025.3
3,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Ballard (County),0.5,8.9,4.4,13.3,0.0,0.0,198.0,25.6,1002.9
4,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Breckinridge (County),0.5,8.9,4.4,13.3,0.0,0.0,198.0,25.6,1002.9
5,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Carlisle (County),0.5,8.9,4.4,13.3,0.0,0.0,198.0,25.6,1002.9
6,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Livingston (County),0.5,8.9,4.4,13.3,0.0,0.0,198.0,25.6,1002.9
7,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Spencer (County),0.5,8.9,4.4,13.3,0.0,0.0,198.0,25.6,1002.9
8,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Hancock (County),0.5,8.9,4.4,13.3,0.0,0.0,198.0,25.6,1002.9
9,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Henderson (County),0.5,8.9,4.4,13.3,0.0,0.0,198.0,25.6,1002.9


In [218]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1940032 entries, 0 to 1940031
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   state            object 
 1   category         object 
 2   date             object 
 3   disaster         object 
 4   disaster_info    object 
 5   designated_area  object 
 6   severity         float64
 7   prcp             float64
 8   pres             float64
 9   snow             float64
 10  tavg             float64
 11  tmax             float64
 12  tmin             float64
 13  wdir             float64
 14  wspd             float64
dtypes: float64(9), object(6)
memory usage: 222.0+ MB


In [232]:
df.to_csv('C:/Users/Phantom/Documents/AI/FloodGuard/final.csv.gz', index=False,compression='gzip')